In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('temperature_data_vif.csv', index_col=0)
data

,u_d,u_q,i_d,pm,motor_speed
0,0.327935,-1.297858,1.029572,-2.522071,-1.222428
1,0.329665,-1.297686,1.029509,-2.522418,-1.222429
2,0.332771,-1.301822,1.029448,-2.522673,-1.222428
3,0.333700,-1.301852,1.032845,-2.521639,-1.222430
4,0.335206,-1.303118,1.031807,-2.521900,-1.222429
...,...,...,...,...,...
998065,0.331475,-1.246114,1.029142,0.429853,-1.222428
998066,0.331701,-1.250655,1.029148,0.429751,-1.222437
998067,0.330946,-1.246852,1.029191,0.429439,-1.222430
998068,0.330987,-1.249505,1.029147,0.429558,-1.222432


In [3]:
X = data[['u_d', 'u_q', 'i_d', 'pm']]
y = data[['motor_speed']]

------------------

Before training the model it would be better to find the proper hyperparameters that would gives the best possible results. So tuning of different hyperparameters is needed before training the model. These parameters are batch size, number of epochs, drop out rate, learning rate, number of neurons in hidden layers, the activation function and the kernel initializer.

#### Tuning of Hyperparameters :- Batch Size and Epochs

In [4]:
# Importing the necessary packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam

In [5]:
# create model
# kernel_initializer is the weight initializer
# linear or step activation for regression problems.
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=4, kernel_initializer='random_uniform', activation='relu'))
    model.add(Dense(8, kernel_initializer='random_uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='random_uniform', activation='linear'))
    
    adam=Adam(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer=adam, metrics=['MeanSquaredError'])
    return model

In [6]:
# Create the model
model = KerasRegressor(build_fn = create_model,verbose = 0)
# Define the grid search parameters
batch_size = [512,1024,2048]
epochs = [5,10]
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,y) 

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START batch_size=512, epochs=5....................................
[CV 1/5; 1/6] END ....batch_size=512, epochs=5;, score=-0.026 total time=  12.7s
[CV 2/5; 1/6] START batch_size=512, epochs=5....................................
[CV 2/5; 1/6] END ....batch_size=512, epochs=5;, score=-0.016 total time=  13.3s
[CV 3/5; 1/6] START batch_size=512, epochs=5....................................
[CV 3/5; 1/6] END ....batch_size=512, epochs=5;, score=-0.011 total time=  13.5s
[CV 4/5; 1/6] START batch_size=512, epochs=5....................................
[CV 4/5; 1/6] END ....batch_size=512, epochs=5;, score=-0.004 total time=  13.6s
[CV 5/5; 1/6] START batch_size=512, epochs=5....................................
[CV 5/5; 1/6] END ....batch_size=512, epochs=5;, score=-0.003 total time=  13.8s
[CV 1/5; 2/6] START batch_size=512, epochs=10...................................
[CV 1/5; 2/6] END ...batch_size=512, epochs=10;, 

In [7]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : -0.008683051588013769, using {'batch_size': 512, 'epochs': 10}
-0.011834447225555777,0.008350022550520704 with: {'batch_size': 512, 'epochs': 5}
-0.008683051588013769,0.005499599216393214 with: {'batch_size': 512, 'epochs': 10}
-0.009858888247981668,0.005435383989254319 with: {'batch_size': 1024, 'epochs': 5}
-0.010344558907672764,0.008101558724919694 with: {'batch_size': 1024, 'epochs': 10}
-0.009851982351392508,0.004721878318032656 with: {'batch_size': 2048, 'epochs': 5}
-0.009914810350164772,0.006766455876685931 with: {'batch_size': 2048, 'epochs': 10}


#### Tuning of Hyperparameters:- Learning rate and Drop out rate

In [8]:
from keras.layers import Dropout

# Defining the model

def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(12,input_dim = 4,kernel_initializer = 'random_uniform',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(8,input_dim = 12,kernel_initializer = 'random_uniform',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'linear'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'mean_squared_error',optimizer = adam,metrics = ['MeanSquaredError'])
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 512,epochs = 10)

# Define the grid search parameters

learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]

# Make a dictionary of the grid search parameters

param_grids = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 1/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.015 total time=  25.5s
[CV 2/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 2/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.009 total time=  25.4s
[CV 3/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 3/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.009 total time=  26.7s
[CV 4/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 4/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.003 total time=  26.7s
[CV 5/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 5/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=-0.004 total time=  26.3s
[CV 1/5; 2/9] START dropout_rate=0.0, learning_rate=0.01........................
[CV 

In [9]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : -0.007851420575752855, using {'dropout_rate': 0.0, 'learning_rate': 0.001}
-0.007851420575752855,0.004347933231515887 with: {'dropout_rate': 0.0, 'learning_rate': 0.001}
-0.00905276513658464,0.007631765482585747 with: {'dropout_rate': 0.0, 'learning_rate': 0.01}
-0.012499921768903733,0.00999416652575866 with: {'dropout_rate': 0.0, 'learning_rate': 0.1}
-0.018090776912868024,0.01473421836682398 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
-0.016995217837393283,0.012932525137396894 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
-0.02801370732486248,0.010593028745082755 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
-0.03012992814183235,0.017992747982672027 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
-0.02701311483979225,0.016939364235333272 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
-0.06399466469883919,0.034590269789329935 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}


#### Tuning of Hyperparameters:- Activation Function and Kernel Initializer

In [10]:
# Defining the model

def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(12,input_dim = 4,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.0))
    model.add(Dense(8,input_dim = 12,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.0))
    model.add(Dense(1,activation = 'linear'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'mean_squared_error',optimizer = adam,metrics = ['MeanSquaredError'])
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 512,epochs = 10)

# Define the grid search parameters
activation_function = ['selu','relu','step','linear']
init = ['uniform','normal','zero']

# Make a dictionary of the grid search parameters
param_grids = dict(activation_function = activation_function,init = init)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START activation_function=selu, init=uniform.....................
[CV 1/5; 1/12] END activation_function=selu, init=uniform;, score=-0.016 total time=  25.8s
[CV 2/5; 1/12] START activation_function=selu, init=uniform.....................
[CV 2/5; 1/12] END activation_function=selu, init=uniform;, score=-0.012 total time=  27.9s
[CV 3/5; 1/12] START activation_function=selu, init=uniform.....................
[CV 3/5; 1/12] END activation_function=selu, init=uniform;, score=-0.011 total time=  27.5s
[CV 4/5; 1/12] START activation_function=selu, init=uniform.....................
[CV 4/5; 1/12] END activation_function=selu, init=uniform;, score=-0.004 total time=  27.2s
[CV 5/5; 1/12] START activation_function=selu, init=uniform.....................
[CV 5/5; 1/12] END activation_function=selu, init=uniform;, score=-0.006 total time=  28.0s
[CV 1/5; 2/12] START activation_function=selu, init=normal................

[CV 4/5; 10/12] END activation_function=linear, init=uniform;, score=-0.042 total time=  29.2s
[CV 5/5; 10/12] START activation_function=linear, init=uniform..................
[CV 5/5; 10/12] END activation_function=linear, init=uniform;, score=-0.041 total time=  27.4s
[CV 1/5; 11/12] START activation_function=linear, init=normal...................
[CV 1/5; 11/12] END activation_function=linear, init=normal;, score=-0.123 total time=  28.6s
[CV 2/5; 11/12] START activation_function=linear, init=normal...................
[CV 2/5; 11/12] END activation_function=linear, init=normal;, score=-0.083 total time=  28.2s
[CV 3/5; 11/12] START activation_function=linear, init=normal...................
[CV 3/5; 11/12] END activation_function=linear, init=normal;, score=-0.077 total time=  26.8s
[CV 4/5; 11/12] START activation_function=linear, init=normal...................
[CV 4/5; 11/12] END activation_function=linear, init=normal;, score=-0.042 total time=  26.4s
[CV 5/5; 11/12] START activat

In [11]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : -0.007323959330096841, using {'activation_function': 'relu', 'init': 'uniform'}
-0.009742717677727341,0.00449674045541203 with: {'activation_function': 'selu', 'init': 'uniform'}
-0.010943353828042745,0.006818618950282039 with: {'activation_function': 'selu', 'init': 'normal'}
-1.1218641877174378,0.1852271955745757 with: {'activation_function': 'selu', 'init': 'zero'}
-0.007323959330096841,0.003228909833627753 with: {'activation_function': 'relu', 'init': 'uniform'}
-0.009559993166476488,0.007477997804787833 with: {'activation_function': 'relu', 'init': 'normal'}
-1.1247742652893067,0.18838606086418064 with: {'activation_function': 'relu', 'init': 'zero'}
nan,nan with: {'activation_function': 'step', 'init': 'uniform'}
nan,nan with: {'activation_function': 'step', 'init': 'normal'}
nan,nan with: {'activation_function': 'step', 'init': 'zero'}
-0.07215921357274055,0.027863309503875628 with: {'activation_function': 'linear', 'init': 'uniform'}
-0.07330212816596031,0.03046825991750

In [13]:
# Defining the model

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 4,kernel_initializer = 'uniform',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(1,activation = 'linear'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'mean_squared_error',optimizer = adam,metrics = ['MeanSquaredError'])
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 512,epochs = 10)

# Define the grid search parameters

neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,y) 

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 1/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.035 total time=  25.0s
[CV 2/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 2/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.023 total time=  26.0s
[CV 3/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 3/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.017 total time=  26.7s
[CV 4/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 4/5; 1/9] END ........neuron1=4, neuron2=2;, score=-0.008 total time=  26.9s
[CV 5/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 5/5; 1/9] END ........neuron1=4, neuron2=2;, score=-1.267 total time=  27.1s
[CV 1/5; 2/9] START neuron1=4, neuron2=4........................................
[CV 1/5; 2/9] END ........neuron1=4, neuron2=4;, 

In [14]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param)) 

Best : -0.007431094627827406, using {'neuron1': 16, 'neuron2': 8}
-0.2700413757935166,0.49867164383993834 with: {'neuron1': 4, 'neuron2': 2}
-0.2675015553832054,0.4986998657720065 with: {'neuron1': 4, 'neuron2': 4}
-0.012919394019991159,0.007862855082191896 with: {'neuron1': 4, 'neuron2': 8}
-0.012671103328466415,0.009311477078626867 with: {'neuron1': 8, 'neuron2': 2}
-0.014470367599278689,0.011279147271103395 with: {'neuron1': 8, 'neuron2': 4}
-0.01093575549311936,0.0073980470892998015 with: {'neuron1': 8, 'neuron2': 8}
-0.009496532939374447,0.005273626039109796 with: {'neuron1': 16, 'neuron2': 2}
-0.009190132981166244,0.00470936584449097 with: {'neuron1': 16, 'neuron2': 4}
-0.007431094627827406,0.0038543359770113294 with: {'neuron1': 16, 'neuron2': 8}


#### Training model with optimum values of Hyperparameters

In [15]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, mean_squared_error, r2_score

# Defining the model

def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim = 4,kernel_initializer = 'uniform',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(8,input_dim = 4,kernel_initializer = 'uniform',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(1,activation = 'linear'))
    
    adam = Adam(lr = 0.001) #sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss = 'mean_squared_error',optimizer = adam,metrics = ['MeanSquaredError'])
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 0,batch_size = 512,epochs = 10)

# Fitting the model

model.fit(X,y)

# Predicting using trained model

y_predict = model.predict(X)

# Printing the metrics
print(mean_squared_error(y, y_predict))
print(r2_score(y, y_predict))

0.0054435335435230135
0.9950255083370564
